In [15]:
SUPPORT_THRESHOLD = 2.9 # To avoid the mistake of using > instead of >=


dataset = """T1		 A,	B,	C,	D,	E,	F
T2		 B,	C,	D,	E,	F,	G
T3		 A,	D,	E,	H
T4		 A,	D,	F,	I,	J
T5		 B,	D,	E,	K""".replace("\t","")

transactions = []
items = {}

from collections import namedtuple
Itemset = namedtuple('Itemset', 'items transactions hv support')

def symbol(transaction):
    return [item if item in transaction else '?' for item in items.keys()]
    

for line in dataset.split("\n"):
    i, stuff = line.split()
    transaction = set(stuff.split(','))
    for item in transaction:
        l = items.get(item,[])
        l.append(str(i))
        items[item] = l
        
    transactions.append(transaction)
 
# Create CSV
print ",".join(items.keys())
for t in transactions:
    print ",".join(symbol(t))


A,C,B,E,D,G,F,I,H,K,J
A,C,B,E,D,?,F,?,?,?,?
?,C,B,E,D,G,F,?,?,?,?
A,?,?,E,D,?,?,?,H,?,?
A,?,?,?,D,?,F,I,?,?,J
?,?,B,E,D,?,?,?,?,K,?


Create candidates, 

In [16]:

all_single_itemsets = {}

for item, trans in items.items():
    support = len(trans) # cheaper to store than call len(transactions?)
    
    if support > SUPPORT_THRESHOLD:

        hv = "".join(item) # sets are unhashable, could make custom set

        itemset = Itemset(items = {item}, transactions = set(trans), hv=hv, support=support)
        all_single_itemsets[hv] = itemset
    
    
 

values = sorted(all_single_itemsets.values(), key=lambda itemset: -itemset.support)
ordered = {k.items.pop() :v for v,k in enumerate(values)}

print ordered

{'A': 2, 'B': 3, 'E': 1, 'D': 0, 'F': 4}


In [21]:
Node = namedtuple('Node', 'item support children next_item')
head = Node(item=None, support={}, children = {}, next_item=None)

table = {k:None for k in ordered.keys()}

print table

for t in transactions:
    # FILTER THOSE BELOW SUPPORT
    print t
    t = [x for x in t if ordered.get(x, False)]
    ordered_transaction = sorted(t, key=lambda x: ordered[x])
    print ordered_transaction
    parent = head
    
    
    for item in ordered_transaction:
        
        child = parent.children.get(item, None)
        
        if child:
            child.support['s'] += 1
        else:
            new_item = Node(item = item, support = {'s':1}, children = {}, next_item=None)
            parent.children[item] = new_item
            child = new_item
            
            ref = table[item]
            prev = None
            if not ref:
                table[item] = ref
            else:
                while ref:
                    prev = ref
                    ref = ref.next_item
            
            
            
        parent = child

    print "\n\n\n\n", head

{'A': None, 'B': None, 'E': None, 'D': None, 'F': None}
set(['A', 'C', 'B', 'E', 'D', 'F'])
['E', 'A', 'B', 'F']




Node(item=None, support={}, children={'E': Node(item='E', support={'s': 1}, children={'A': Node(item='A', support={'s': 1}, children={'B': Node(item='B', support={'s': 1}, children={'F': Node(item='F', support={'s': 1}, children={}, next_item=None)}, next_item=None)}, next_item=None)}, next_item=None)}, next_item=None)
set(['C', 'B', 'E', 'D', 'G', 'F'])
['E', 'B', 'F']




Node(item=None, support={}, children={'E': Node(item='E', support={'s': 2}, children={'A': Node(item='A', support={'s': 1}, children={'B': Node(item='B', support={'s': 1}, children={'F': Node(item='F', support={'s': 1}, children={}, next_item=None)}, next_item=None)}, next_item=None), 'B': Node(item='B', support={'s': 1}, children={'F': Node(item='F', support={'s': 1}, children={}, next_item=None)}, next_item=None)}, next_item=None)}, next_item=None)
set(['A', 'H', 'E', 'D'])
['E', 'A']




Node(item=

# I messed UP!
I combined a top down with a bottum-up approach. It seem to get the support you should use the table that links nodes to traverse (done) but I don't have access to parents. If you did have access to parents you could get the powerset of all ancestors and for each set in the powerset get the hash value (by joining items). In python collections there is a counter collection that can be created from a dictionary. So at each node creat a hash value-support counter and sum all of these together.


In [18]:
#Depth-First Traversal
nodes = []

for n in head.children.values():
        nodes.append([n])

while nodes:
    node_list = nodes.pop()
    print ",".join([x.item for x in node_list[:-1]]),"=>",node_list[-1].item, "Support =", node_list[-1].support['s']
    node = node_list[-1]
    for n in node.children.values():
        nodes.append(node_list + [n])
        
print "\n\nWith Support and Confidence\n\n"

SUPPORT_THRESHOLD = 1.9

for n in head.children.values():
        nodes.append(([n], None))
        0
print "LEN", len(nodes)

while nodes:
    node_list, parent_support = nodes.pop()
    end_node = node_list[-1]
    if parent_support and end_node.support['s'] > SUPPORT_THRESHOLD:
        lineA = ",".join([x.item for x in node_list[:-1]]),"=>",node_list[-1].item, "Support =", node_list[-1].support['s']
        lineB = "Confidence = ", 1.0*node_list[-1].support['s']/all_single_itemsets[end_node.item].support
        print lineA, lineB
    node = node_list[-1]
    for n in node.children.values():
        nodes.append((node_list + [n], end_node.support))

 => E Support = 4
E => B Support = 2
E,B => F Support = 1
E => A Support = 2
E,A => B Support = 1
E,A,B => F Support = 1
 => A Support = 1
A => F Support = 1


With Support and Confidence


LEN 2
('E', '=>', 'B', 'Support =', 2) ('Confidence = ', 0.6666666666666666)
('E', '=>', 'A', 'Support =', 2) ('Confidence = ', 0.6666666666666666)
